#Data Parsing, Cleansing and Integration
## Task 3
#### Student Name: Matthew Bentham
#### Student ID: s3920376

Date: 28/08/2022

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: 
* pandas
* re
* numpy
* datetime
* random


## Introduction

The objective of this task (Task 3) is to combine the data outputted from task 2 of this assement and successfully intergrate at both the schema and data level with data obtained from www.jobhuntlisting.com. To do so the schema of the task 2 dataset will be adopted as the global schema. 

This process can be brocken down into two steps: 

1. **Resolving schema level conflicts:** The process and conflicts found are layed out in section 2 of this report.
2. **Resolving data level conflicts**: invloves    
a) Resolving and complete duplications in unified table     
b) Identifying any unique keys for the intergrated data and removing any datapoints that breech these unique contraints


##  Import libraries 

In [28]:
# Code to import libraries as you need in this assessment
import numpy as np
import pandas as pd
import datetime
import re
from random import randint

### 1. Examining and loading data
Examine "s3923076_dataset2.csv" the structure and schema. 

In [29]:
pd.set_option('display.float_format', '{:.2f}'.format)
df1 = pd.read_csv('s3923076_dataset1_solution.csv')
df2 = pd.read_csv('s3923076_dataset2.csv')
# Display two dataframes together to compare
print('dataset 1:' + str(df1.shape))

display(df1.head())
print('dataset 2:' + str(df2.shape))
display(df2.head())

dataset 1:(48244, 11)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,71226442,Accounts assistant Wilmslow,Stockport,RK Accountancy,full_time,contract,Accounting & Finance Jobs,12960.00,2012-10-29 00:00:00,2013-01-27 00:00:00,fish4.co.uk
1,69219070,"Web Developer ( HTML, CSS, NET )",Bournemouth,IT Executive Recruitment,full_time,permanent,IT Jobs,28500.00,2013-01-05 12:00:00,2013-04-05 12:00:00,fish4.co.uk
2,69205773,Solutions Architect MS Dynamics CRM,UK,Syntony Search and Selection Ltd,full_time,permanent,IT Jobs,60000.00,2013-10-21 12:00:00,2013-11-20 12:00:00,fish4.co.uk
3,68509292,Head of Policy and Committee Secretariat,Wales,non-specified,full_time,contract,Healthcare & Nursing Jobs,42558.00,2013-04-18 00:00:00,2013-06-17 00:00:00,fish4.co.uk
4,62114107,"Business Analysts Manchester, Liverpool, Shef...",London,Additional Resources,full_time,permanent,IT Jobs,45000.00,2012-12-30 12:00:00,2013-02-28 12:00:00,fish4.co.uk


dataset 2:(5000, 9)


,Location,Job Title,Monthly Payment,Closing,Category,Type,Opening,Organisation,Full-Time Equivalent (FTE)
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,NaN,2012-01-08 12:00:00,SEC Recruitment,1.00
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,Permanent,2013-08-07 12:00:00,Navartis Limited,1.00
2,Coventry,NVQ Assessor Banking/Financial Services Salary...,1791.67,2013-05-02 00:00:00,Hospitality,Permanent,2013-02-01 00:00:00,Pertemps,1.00
3,Poole,HEAD OF CARE (RGN/RMN) Poole,2916.67,2013-09-24 15:00:00,Health,NaN,2013-07-26 15:00:00,Team 24 Ltd,1.00
4,Woking,SQL Server DBA,3125.00,2012-12-21 12:00:00,Information Technology,NaN,2012-11-21 12:00:00,Matchtech,1.00


In [30]:
# Check datatypes: 
df2.dtypes


Location                       object
Job Title                      object
Monthly Payment               float64
Closing                        object
Category                       object
Type                           object
Opening                        object
Organisation                   object
Full-Time Equivalent (FTE)    float64
dtype: object

In [31]:
# Locate missing values:
df2.isna().sum()

Location                         0
Job Title                        0
Monthly Payment                  0
Closing                          0
Category                         0
Type                          1407
Opening                          0
Organisation                   469
Full-Time Equivalent (FTE)       0
dtype: int64

### 2. Resolving schema conflicts:

In order to intergrate the two job listing datasets , the schema of dataset 1 will be adopted as the global schema to resolve schema conflicts incrementally. 

**Global Schema (dataset 1)**:   
| Feature | Format and domain values |
| --- | --- |
| `Id` | 8 digit integer that as an unique identifier for each job. |
| `Title` | Variable length string for job title. 'non-specified' when no value is given. |
| `Location` | Variable length string for Location. 'non-specified' when no value is given|
| `Company` | Variable length string for Company name. 'non-specified' when no value is given |
| `ContractType` | String was a fixed value of either ‘full_time’, ‘part_time’ or ‘non-specified’ denoting position type. |
| `ContractTime` | String was a fixed value of either  ‘permanent’, ‘contract’ or ‘non-specified’ denoting employment type. |
| `Category` | String that fits into the eight job categories  |
| `Salary` | non-null float (2d.p) denoting annual salary |
| `OpenDate` | Open date of job listing in yyyy-mm-dd hh:mm:ss datetime format |
| `CloseDate` | Close date of job listing in yyyy-mm-dd hh:mm:ss datetime format |
| `SourceName` | Variable length string for source of data. 'non-specified' when no value is given. |

**Schema Conflicts found and resolutions**:   
| Feature | Schema conflict | Resolution |
| --- | --- | --- |
| `Id` | No Id column present | Generate unique ids for job listings in dataset 2 without altering dataset 1|
| `Title` | Name of column | Change name to match schema |  
| `Company` | Missing values present , Varying column name | convert missing values to 'non-specified' and change name to match schema | 
| `ContractType` | Dataset 2 does not have ContractType column  | ContractType can be calculated from FTE column |
| `ContractTime` | Domain values seems dont meet the global schema requirement | Convert domain values to the correct schema counter-part |
| `Category` | Domain values seems dont meet the global schema requirement | Convert domain values to the correct schema counter-part |
| `Salary` | Unit values dont meet schema requirments | Need to calculate per annum from monthly values |
| `SourceName` | Column is non-existent in dataset2 | treat every row as having a 'non-specified' SourceName|



### 2.1 Schema conflict: `Id` column

It seems as though dataset 2 has no unique identifier for each job listing (Id). In this section , we need to generate an Id for each job listing in dataset 2 that: 
- Is unique for every indivdual job listing 
- Doesnt conflict with any Ids in dataset 1 

In [32]:
# list of existing Ids in dataset 1
existing_ids = df1.Id.to_list()

def Generate_ids(dataframe,forbiden_ids):
    """ Function to generate unique 8 digit values for a dataframe that does not conflict with 
    forbiden_ids
    """
    Id = randint(10000000,99999999) # randomly generates an 8 digit number 
    Ids = [] 
    for i in range(0,dataframe.shape[0]): 
        w= 0
        while w == 0: 
            if Id in forbiden_ids: # add 1 to id until it doesnt conflict with forbiden_ids
                Id += 1 
            else:
                w=1
        Ids.append(Id)
        Id += 1
    dataframe['Id'] = Ids # create Id column 

In [33]:
Generate_ids(df2,existing_ids)
df2.head(5)

,Location,Job Title,Monthly Payment,Closing,Category,Type,Opening,Organisation,Full-Time Equivalent (FTE),Id
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,NaN,2012-01-08 12:00:00,SEC Recruitment,1.00,96644269
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,Permanent,2013-08-07 12:00:00,Navartis Limited,1.00,96644270
2,Coventry,NVQ Assessor Banking/Financial Services Salary...,1791.67,2013-05-02 00:00:00,Hospitality,Permanent,2013-02-01 00:00:00,Pertemps,1.00,96644271
3,Poole,HEAD OF CARE (RGN/RMN) Poole,2916.67,2013-09-24 15:00:00,Health,NaN,2013-07-26 15:00:00,Team 24 Ltd,1.00,96644272
4,Woking,SQL Server DBA,3125.00,2012-12-21 12:00:00,Information Technology,NaN,2012-11-21 12:00:00,Matchtech,1.00,96644273


### 2.2 Schema conflict: `Title` vs `Job Title` column

Only obvious schema conflict seems to be the differing column names 

In [34]:
print('Number of missing values:', df2['Job Title'].isna().sum())

Number of missing values: 0


In [35]:
df2 = df2.rename(columns={'Job Title': 'Title'})
df2.head(2)

,Location,Title,Monthly Payment,Closing,Category,Type,Opening,Organisation,Full-Time Equivalent (FTE),Id
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,NaN,2012-01-08 12:00:00,SEC Recruitment,1.00,96644269
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,Permanent,2013-08-07 12:00:00,Navartis Limited,1.00,96644270


### 2.3 Schema conflict: `Company` vs `Organisation` column
Missing values have conflicting domain values specified by the global schema 

In [36]:
print('Number of missing values:',df2.Organisation.isna().sum())

Number of missing values: 469


In [37]:
# fill missing vales with 'non-specified'
df2['Organisation']=df2['Organisation'].fillna('non-specified')
#rename column
df2 = df2.rename(columns={'Organisation': 'Company'})
df2.head(2)

,Location,Title,Monthly Payment,Closing,Category,Type,Opening,Company,Full-Time Equivalent (FTE),Id
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,NaN,2012-01-08 12:00:00,SEC Recruitment,1.00,96644269
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,Permanent,2013-08-07 12:00:00,Navartis Limited,1.00,96644270


### 2.3 Schema conflict: `ContractType` vs `Full-Time Equivalent(FTE)` column
Dataset 2 doesnt seem to have any values that correspond to contract type however this can be calculated from the Full-Time Equivalent column. FTE is an employee's scheduled hours divided by the employer's hours for a full-time workweek , there an FTE of 1.0 means the job is full-time and and FTE < 1.0 means the job is part-time. 

In [38]:
df2['Full-Time Equivalent (FTE)'].value_counts()

1.00    4857
0.20      39
0.80      36
0.60      36
0.40      32
Name: Full-Time Equivalent (FTE), dtype: int64

In [39]:
def Contracttype(num):
    """Converts FTE value to its ContractType equivalent 
    """
    if num == 1.0:
        string = 'full_time'
    elif num < 1.0:
        string = 'part-time'
    else:
        string = 'non-specified'
    return string
# Change domain values to global schema equivalent 
df2['Full-Time Equivalent (FTE)'] = df2['Full-Time Equivalent (FTE)'].apply(lambda x: Contracttype(x))
# Rename column: 
df2 = df2.rename(columns={'Full-Time Equivalent (FTE)': 'ContractType'})
df2.head(2)

,Location,Title,Monthly Payment,Closing,Category,Type,Opening,Company,ContractType,Id
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,NaN,2012-01-08 12:00:00,SEC Recruitment,full_time,96644269
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,Permanent,2013-08-07 12:00:00,Navartis Limited,full_time,96644270


### 2.4 Schema conflict: `ContractTime` vs `Type` column
The type column in dataset 2 contains the domain values: 
- Permanent 
- Fixed term contract 
- NaN  

as appose to the values laid out in the global schema: 
- permanent
- contract 
- non-specified


In [40]:
df2.Type.value_counts()


Permanent              3034
Fixed Term Contract     559
Name: Type, dtype: int64

In [41]:
print('Number of missing values:',df2.Type.isna().sum())

Number of missing values: 1407


In [42]:
# replace missing values with non-specified 
df2['Type']=df2['Type'].fillna('non-specified')
# replace Permanent and Fixed term Contract with correct values 
df2=df2.replace('Permanent', 'permanent')
df2=df2.replace('Fixed Term Contract', 'contract')
# Rename column to match schema 
df2 = df2.rename(columns={'Type': 'ContractTime'})
df2.head(2)

,Location,Title,Monthly Payment,Closing,Category,ContractTime,Opening,Company,ContractType,Id
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,non-specified,2012-01-08 12:00:00,SEC Recruitment,full_time,96644269
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,permanent,2013-08-07 12:00:00,Navartis Limited,full_time,96644270


### 2.5 Schema conflict: `Category` columns
The Category column in dataset 2 contains the domain values: 
- 'Health'
- 'Engineering'
- 'Hospitality'
- 'Information Technology',
- 'Finance'
- 'Marketing'
- 'Sales' 
- 'Education'

as appose to the values laid out in the global schema: 
- 'Accounting & Finance Jobs'
- 'IT Jobs'
- 'Healthcare & Nursing Jobs'
- 'PR, Advertising & Marketing Jobs'
- 'Engineering Jobs'
- 'Hospitality & Catering Jobs'
- 'Teaching Jobs',
- 'Sales Jobs'

In [43]:
print('Number of missing values:',df2.Category.isna().sum())

Number of missing values: 0


In [44]:
### 2.3 Schema conflict: `Category` columns
df1cats = ['Healthcare & Nursing Jobs','Engineering Jobs','Hospitality & Catering Jobs', 'IT Jobs','Accounting & Finance Jobs','PR, Advertising & Marketing Jobs','Sales Jobs'
        ,'Teaching Jobs']
df2cats = ['Health', 'Engineering', 'Hospitality', 'Information Technology','Finance', 'Marketing', 'Sales', 'Education']
print('Dataset 2 categories:\n',df2cats,'\n')
print('Dataset 1 categories:\n',df1cats)


Dataset 2 categories:
 ['Health', 'Engineering', 'Hospitality', 'Information Technology', 'Finance', 'Marketing', 'Sales', 'Education'] 

Dataset 1 categories:
 ['Healthcare & Nursing Jobs', 'Engineering Jobs', 'Hospitality & Catering Jobs', 'IT Jobs', 'Accounting & Finance Jobs', 'PR, Advertising & Marketing Jobs', 'Sales Jobs', 'Teaching Jobs']


In [45]:
# Code to fix the conflict
for i,cat in enumerate(df2cats):
    df1cats[i]
    df2=df2.replace(cat, df1cats[i])
print(df2.Category.value_counts())

IT Jobs                             1251
Healthcare & Nursing Jobs            804
Engineering Jobs                     732
Accounting & Finance Jobs            675
Sales Jobs                           517
Hospitality & Catering Jobs          436
Teaching Jobs                        341
PR, Advertising & Marketing Jobs     244
Name: Category, dtype: int64


### 2.6 Schema conflict: `Salary` vs `Monthly Payments` columns
dataset 2 seems to have the job list salary in $/month units as appose to per annum as specified in the global schema 

In [46]:
print('Number of missing values:',df2['Monthly Payment'].isna().sum())

Number of missing values: 0


In [47]:
# Convert monthly payment to per year:
df2['Monthly Payment'] = df2['Monthly Payment'].apply(lambda x: x*12)
# Rename column 
df2 = df2.rename(columns={'Monthly Payment': 'Salary'})
df2.head(2)

,Location,Title,Salary,Closing,Category,ContractTime,Opening,Company,ContractType,Id
0,Berkshire,Lead CRA UK,54999.96,2012-03-08 12:00:00,Healthcare & Nursing Jobs,non-specified,2012-01-08 12:00:00,SEC Recruitment,full_time,96644269
1,Bristol,Possession Manager,33750.00,2013-09-06 12:00:00,Engineering Jobs,permanent,2013-08-07 12:00:00,Navartis Limited,full_time,96644270


### 2.7 Schema conflict: lack of `SourceName` column

All data from dataset 2 comes from an individual source: http://www.jobhuntlisting.com/ , therefore so that the columns match the global schema requirements all SoureName values can be inputted as www.jobhuntlisting.com


In [48]:
# insert 'non-specified' as all sourcename values for dataset 2
df2['SourceName'] = ' www.jobhuntlisting.com'
df2.head(2)

,Location,Title,Salary,Closing,Category,ContractTime,Opening,Company,ContractType,Id,SourceName
0,Berkshire,Lead CRA UK,54999.96,2012-03-08 12:00:00,Healthcare & Nursing Jobs,non-specified,2012-01-08 12:00:00,SEC Recruitment,full_time,96644269,www.jobhuntlisting.com
1,Bristol,Possession Manager,33750.00,2013-09-06 12:00:00,Engineering Jobs,permanent,2013-08-07 12:00:00,Navartis Limited,full_time,96644270,www.jobhuntlisting.com


### 2.8 Schema conflict: rename `Closing` and `Opening` columns

In [49]:
df2 = df2.rename(columns={'Opening': 'OpenDate','Closing': 'CloseDate'})
#Reorder columns: 
df2 = df2[['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName']]
df2.head(1)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,96644269,Lead CRA UK,Berkshire,SEC Recruitment,full_time,non-specified,Healthcare & Nursing Jobs,54999.96,2012-01-08 12:00:00,2012-03-08 12:00:00,www.jobhuntlisting.com


## 3. Merge Data

Merging both datasets into one  

In [50]:
#Join datasets 
df = pd.concat([df1,df2])
print (df.shape)
df.sample(2)

(53244, 11)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
18892,68667849,Recruitment Consultant / Account Manager NO CO...,Sheffield,Listen Recruitment,non-specified,permanent,Sales Jobs,23000.00,2013-05-08 00:00:00,2013-08-06 00:00:00,totaljobs.com
71,69216953,Financial Controller Cash Management,Southampton,Byron Finance,full_time,permanent,Accounting & Finance Jobs,48750.00,2012-06-03 12:00:00,2012-07-03 12:00:00,fish4.co.uk


## 4. Resolve data-level conflicts
- Use Pandas libraries to detect and resolve duplications in the unified table.
- Identify a proper global/unique key for the integrated job data and explain your chosen key in the notebook, i.e., why you think the chosen key can be used as a unique identifier of a job
advertisement.

In [51]:
# Check duplications: 
#Duplications including id: 
cols = df.columns.to_list()
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head(10)

#Duplicates not inlcuding id (as dataset 2 were given unique ids)
cols = [ 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName']
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head(10)

There are 0 duplicate records found
There are 0 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


The set attributes that would definte a unique key (aside from job id) for the integrated job data could be: 
- Category type
- Location 
- Company 
- Contract Type 
- Contract Time 
- Salary 
- Opendate 
- CloseDate 

As the combination of these values should be unique for each job listing. This is becuase values like title are subject to differences based on datasource , for example two of the same jobs may be given different names on different cites even though all intrinsic information of the job listing itself is the same.

In [52]:
# check duplication
cols = ['Category','Location', 'Company', 'ContractType', 'ContractTime',
       'Salary', 'OpenDate', 'CloseDate'] # uniquely identified a record.
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head()

There are 66 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
43091,67101243,Accounting Policy Associate,Gloucestershire,CMC Consulting,full_time,permanent,Accounting & Finance Jobs,37500.00,2013-05-26 00:00:00,2013-06-25 00:00:00,GAAPweb
4951,96649220,Accounting Policy Associate,Gloucestershire,CMC Consulting,full_time,permanent,Accounting & Finance Jobs,37500.00,2013-05-26 00:00:00,2013-06-25 00:00:00,www.jobhuntlisting.com
47973,70238578,Contract IT Auditor (6 Months) – Banking,London,Austin Andrew,full_time,permanent,Accounting & Finance Jobs,82500.00,2013-05-28 00:00:00,2013-08-26 00:00:00,careersinaudit.com
1644,96645913,Contract IT Auditor (6 Months) – Banking,London,Austin Andrew,full_time,permanent,Accounting & Finance Jobs,82500.00,2013-05-28 00:00:00,2013-08-26 00:00:00,www.jobhuntlisting.com
16564,68384942,Commercial Analyst – Media/Content,London,SAUCE RECRUITMENT,full_time,permanent,Accounting & Finance Jobs,52500.00,2013-04-08 15:00:00,2013-05-08 15:00:00,jobs.guardian.co.uk


In [53]:
df = df.drop_duplicates(cols, keep='first')

## 5. Save integrated data

Finally, we just save the integrated data into another csv file:

In [54]:
df.to_csv("s3923076_dataset_integrated.csv",float_format='%.2f')

## Summary of the Assessment Task

In summary, in order to successfully merge the two datasets numerous schema level and data level conflicts were detected and resolved to produce a unified table that doesn’t conflict with the global schema. 

In terms of the global key identified, it’s important to note that what makes two jobs listing unique is very specific to the data’s context. For example, category type and title could vary from source to source even for the same job listing , however they are also the only variables that differentiate two jobs at the same company with the same salary , contract and posting dates, therefore more information is needed on the data to make a better informed descion. 
